In [30]:
# import torch
# from torch.nn import Linear, Parameter
# from torch_geometric.nn import MessagePassing
# from torch_geometric.utils import add_self_loops, degree

# class GCNConv(MessagePassing):
#     def __init__(self, in_channels, out_channels):
#         super().__init__(aggr='add')  # "Add" aggregation (Step 5).
#         self.lin = Linear(in_channels, out_channels, bias=False)
#         '''
#         nn.Linear(n,m) is a module that creates single layer feed forward network with n inputs and m output. 
#         Mathematically, this module is designed to calculate the linear equation Ax = b where x is input, b is output, A is weight. 
#         This is where the name 'Linear' came from. Essentially calculates weights for the model
#         '''
#         self.bias = Parameter(torch.Tensor(out_channels))

#         self.reset_parameters()

#     def reset_parameters(self):
#         self.lin.reset_parameters()
#         self.bias.data.zero_()

#     def forward(self, x, edge_index):
#         # x has shape [N, in_channels]
#         # edge_index has shape [2, E]

#         # Step 1: Add self-loops to the adjacency matrix.
#         edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

#         # Step 2: Linearly transform node feature matrix.
#         x = self.lin(x)

#         # Step 3: Compute normalization.
#         row, col = edge_index
#         deg = degree(col, x.size(0), dtype=x.dtype)
#         deg_inv_sqrt = deg.pow(-0.5)
#         deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
#         norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

#         # Step 4-5: Start propagating messages.
#         out = self.propagate(edge_index, x=x, norm=norm)

#         # Step 6: Apply a final bias vector.
#         out += self.bias

#         return out

#     def message(self, x_j, norm):
#         # x_j has shape [E, out_channels]

#         # Step 4: Normalize node features.
#         return norm.view(-1, 1) * x_j

X is the feature vector i.e training data. Each node in the graph has a feature vector and 

In [ ]:
# import torch.nn.functional as F
# from torch_geometric.nn import GCNConv
# import torch

# class GCN(torch.nn.Module):
#     def __init__(self, hidden_channels):
#         super(GCN, self).__init__()
#         torch.manual_seed(12345)
#         self.conv1 = GCNConv(1, hidden_channels)
#         self.conv2 = GCNConv(hidden_channels, dataset.num_classes)
#         self.linear1 = torch.nn.Linear(100,1)
#     def forward(self, x, edge_index):
#         x = self.conv1(x, edge_index)
#         x = x.relu()
#         x = F.dropout(x, p=0.5, training=self.training)
#         x = self.conv2(x, edge_index)
#         x = self.linear1(x) # This probably isnt needed just return x
#         return x

In [ ]:
import torch
from torch_geometric.data import Data
from glob import glob
import xarray as xr

edge_index = []
x = []

for i in range(39):
    for j in range(39):
        if i != j:
            edge_index.append([i, j])

edge_index = torch.tensor(edge_index, dtype=torch.long)

for filename in glob.glob('data\\tas_scenario_245\\tas_mon_mod_ssp245_192_*.nc'):
    print('Processing', filename)
    
    data = xr.open_dataset(filename)
    tas_vector = data.to_dataframe().reset_index()['tas']
    if filename == 'data\\tas_scenario_245\\tas_mon_mod_ssp245_192_000.nc':
        print('y found')
        y = tas_vector
    else:
        x.append(tas_vector)

x = torch.tensor(x, dtype=torch.float)
y = torch.tensor(y, dtype=torch.float)
data = Data(x=x, edge_index=edge_index.t().contiguous(), y=y)

In [29]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import torch

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.edge_weight = torch.nn.Parameter(torch.ones(data.num_edges))
        self.conv1 = GCNConv(data.num_node_features, 16)
        self.conv2 = GCNConv(16, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index,torch.minimum(self.edge_weight.abs(),torch.ones(data.num_edges)))
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index,torch.minimum(self.edge_weight.abs(),torch.ones(data.num_edges)))

        return x


In [31]:
data.validate(raise_on_error=True)

True

In [34]:
data.view()

AttrView(store=Data(x=[39, 83275776], edge_index=[2, 1482], y=[83275776]), attr=DataTensorAttr(group_name=None, attr_name=<FieldStatus.UNSET: 1>, index=None))

In [27]:
data.num_node_features

83275776

In [33]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = data.to(device)


optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(100):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out, data.y)
    loss.backward()
    optimizer.step()

ValueError: Expected input batch_size (39) to match target batch_size (83275776).